Ch6. Using a nueral network to fit the data
-------------------------------------------

신경망의 기본 구조는 **뉴런**으로 이루어진 **계층**이다.
뉴런은 입력에 대해 선형변환을 한 뒤 **활성 함수**(**Activation function**)라는 비선형 함수를 적용한다.

이를 일반적인 수식으로 나타내면 $o =f(w * x + b)$으로 나타낼 수 있다. 각 변수가 스칼라 값이 아닌 벡터 값이면 여러 차원으로 가중치와 편향 값을 가진 여러 개의 뉴런으로 나타낸다. 이를 뉴런 **계층**이라고 한다.

In [4]:
%matplotlib inline
import numpy as np
import torch
import torch.optim as optim

torch.set_printoptions(edgeitems=2, linewidth=75)
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c).unsqueeze(1) # <1>
t_u = torch.tensor(t_u).unsqueeze(1) # <1>

n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

t_u_train = t_u[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_u[val_indices]
t_c_val = t_c[val_indices]

t_un_train = 0.1 * t_u_train
t_un_val = 0.1 * t_u_val

랜덤으로 train과 valid를 나누기 때문에 초기 가중치와 편향치가 책과 다를 수 있다.

In [5]:
import torch.nn as nn

linear_model = nn.Linear(1,1) # num of input feature, num of output feature, bias(defualt = True)
linear_model(t_un_val)

tensor([[2.8406],
        [0.5912]], grad_fn=<AddmmBackward0>)

아래 두 줄은 결과적으로 같은 output이지만 `model`에 인자값을 넣고 호출하는 게 안전하다. `nn/modules/module.py`를 보면 `forward`를 진행하기 전에 수많은 `hook`을 진행한다.

In [6]:
y = model(x) # 권장
# y = model.forward(x)

NameError: name 'model' is not defined

In [7]:
linear_model.weight

Parameter containing:
tensor([[0.5827]], requires_grad=True)

In [8]:
linear_model.bias

Parameter containing:
tensor([-0.6792], requires_grad=True)

In [9]:
x = torch.ones(1)
linear_model(x)

tensor([-0.0964], grad_fn=<AddBackward0>)

위 코드에서 `linear_model`은 원하는 값을 뱉었다. 하지만 파이토치의 `nn.Module`은 다양한 샘플을 받기 위해  0번째 차원에 배치(batch)의 수가 들어있는걸 전제로 한다. $B \times Nin$ 크기의 텐서를 넣어보자. $B$는 배치의 사이즈를 나타내고 $Nin$은 input feature의 수를 나타낸다.

In [10]:
x = torch.ones(10, 1)
linear_model(x)

tensor([[-0.0964],
        [-0.0964],
        [-0.0964],
        [-0.0964],
        [-0.0964],
        [-0.0964],
        [-0.0964],
        [-0.0964],
        [-0.0964],
        [-0.0964]], grad_fn=<AddmmBackward0>)

입력한 데이터의 배치 수와 동일한 결과의 수가 나왔다.

> 배치를 사용하는 이유
> - 연산 자원을 최대한 활용하기 위해
> - (특정 모델의 경우) 전체 배치의 통계적 정보를 이용하기 때문. 배치의 사이즈가 클수록 더 좋은 결과를 얻는다.

기존 온도 예제 데이터는 단순한 1차원 형태의 데이터였다. 이를 배치 형태($B \times Nin$)로 바꾸기 위해 `unsqeeze`를 사용한다.

In [11]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c).unsqueeze(1) # <1>
t_u = torch.tensor(t_u).unsqueeze(1) # <1>

t_u.shape

torch.Size([11, 1])

In [12]:
linear_model = nn.Linear(1,1)
optimizer = optim.SGD(
    linear_model.parameters(), # 기존 params 자리
    lr = 1e-2
)

In [13]:
list(linear_model.parameters())

[Parameter containing:
 tensor([[0.1060]], requires_grad=True),
 Parameter containing:
 tensor([-0.6616], requires_grad=True)]

In [14]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,
                  t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train)
        loss_train = loss_fn(t_p_train, t_c_train)

        # 왜 valid 셋에 nograd 안하지?
        t_p_val = model(t_u_val)
        loss_val = loss_fn(t_p_val, t_c_val)

        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()

        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")

In [15]:
training_loop(
    n_epochs=3000,
    optimizer=optimizer,
    model=linear_model,
    loss_fn=nn.MSELoss(),
    t_u_train=t_un_train,
    t_u_val=t_un_val,
    t_c_val=t_c_val,
    t_c_train=t_c_train,
)
print()
print(linear_model.weight)
print(linear_model.bias)

Epoch 1, Training loss 208.1362, Validation loss 91.1518
Epoch 1000, Training loss 4.4426, Validation loss 2.5459
Epoch 2000, Training loss 2.6901, Validation loss 4.2043
Epoch 3000, Training loss 2.5447, Validation loss 5.7748

Parameter containing:
tensor([[5.6493]], requires_grad=True)
Parameter containing:
tensor([-18.7451], requires_grad=True)


이전 챕터에서 한 훈련 방식을 파이토치의 모듈로 간단하게 표현하고 진행했다. 지금부터는 선형 함수 모델에 활성 함수를 씌워 간단한 신경망을 만들어보도록 한다.

지금부터 만들 신경망의 아키텍처는 아래 그림과 같다. 기본적으로 신경망은 선형 모듈에 활성 함수를 씌운 층(layer, module)을 거친 값을 다음 선형 모듈에 넣는 방식이다.

![](.notepad_images/241f3591.png)
